# Semantic Image Segmentation
with the Oxford-IIIT Pet dataset

M. Hawryluk, AGH 2023


## Import modules

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow.keras.utils as utils
import cv2
from random import shuffle, seed

ModuleNotFoundError: No module named 'tensorflow'

## Data preprocessing

In [4]:
IMAGE_WIDTH = 128
IMAGE_HEIGHT = 128

def resize(image):
   return tf.image.resize(image, (IMAGE_WIDTH, IMAGE_HEIGHT), method="nearest")

In [2]:
oxford_ds, oxford_info = tfds.load('oxford_iiit_pet:3.*.*', with_info=True)

NameError: name 'tfds' is not defined

In [3]:
def preprocess_data(data):
   image = data["image"]
   mask = data["segmentation_mask"]

   image = resize(image)
   mask = resize(mask)

   image = tf.cast(image, tf.float32) / 255.0
   mask -= 1

   return image, mask

In [5]:
ds_train = oxford_ds["train"].map(preprocess_data)
ds_test = oxford_ds["test"].map(preprocess_data)

NameError: name 'oxford_ds' is not defined

In [6]:
print("image")
next(iter(ds_train))[0]

image


NameError: name 'ds_train' is not defined

In [ ]:
example_mask = next(iter(ds_train))[1].numpy()

np.unique(example_mask)

In [ ]:
batch_size = 64

ds_train_batched = ds_train.cache().batch(batch_size).repeat()

ds_val_batched = ds_test.take(3000).batch(batch_size)
ds_test_batched = ds_test.skip(3000).batch(batch_size)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 10))

x, y = next(iter(ds_train))

ax[0].axis('off')
ax[1].axis('off')

ax[0].imshow(x)
ax[1].imshow(y)

plt.show()

## Neural segmentation model
### Unet

In [ ]:
def downsample_block(x, n_filters):
   f = layers.Conv2D(n_filters, 3, padding="same", activation="relu")(x)
   p = layers.MaxPool2D(2)(f)
   p = layers.Dropout(0.3)(p)
   return f, p

def upsample_block(x, conv_features, n_filters):
   x = layers.Conv2DTranspose(n_filters, 3, 2, padding="same")(x)
   x = layers.concatenate([x, conv_features])
   x = layers.Dropout(0.3)(x)
   x = layers.Conv2D(n_filters, 3, padding="same", activation="relu")(x)
   return x

def unet(num_classes=3):
   inputs = layers.Input(shape=(IMAGE_WIDTH, IMAGE_HEIGHT, 3))

   # encoder
   f1, p1 = downsample_block(inputs, 64)
   f2, p2 = downsample_block(p1, 128)
   f3, p3 = downsample_block(p2, 256)
   f4, p4 = downsample_block(p3, 512)
   bottleneck = layers.Conv2D(1024, 3, padding = "same", activation = "relu")(p4)

   # decoder
   u6 = upsample_block(bottleneck, f4, 512)
   u7 = upsample_block(u6, f3, 256)
   u8 = upsample_block(u7, f2, 128)
   u9 = upsample_block(u8, f1, 64)
   outputs = layers.Conv2D(num_classes, 1, padding="same", activation="softmax")(u9)

   unet = tf.keras.Model(inputs, outputs)
   return unet

In [ ]:
def get_mask(prediction):
 mask = tf.argmax(prediction, axis=-1)
 mask = mask[..., tf.newaxis]
 return mask[0]

In [ ]:
model = unet()
model.compile(
    optimizer=tf.keras.optimizers.legacy.Adam(),
    loss="sparse_categorical_crossentropy",
    metrics="accuracy"
)

In [ ]:
epochs = 20

model_history = model.fit(
    ds_train_batched,
    epochs=epochs,
    steps_per_epoch=oxford_info.splits["train"].num_examples // batch_size,
    validation_steps=oxford_info.splits["test"].num_examples // batch_size // 5,
    validation_data=ds_val_batched
)

In [7]:
predicted = model.predict(ds_test_batched)

NameError: name 'model' is not defined

In [ ]:
fig, ax = plt.subplots(10, 3, figsize=(13, 30))

for i, (x, y) in enumerate(ds_test.take(10)):
  y_pred = get_mask(model.predict(tf.expand_dims(x, axis=0)))

  ax[i][0].axis('off')
  ax[i][1].axis('off')
  ax[i][2].axis('off')

  ax[i][0].imshow(x)
  ax[i][1].imshow(y)
  ax[i][2].imshow(y_pred)

plt.show()

In [ ]:
model.save('pets-1')